# Handout 07
#### Sara Díaz del Ser

In [1]:
import matplotlib.pyplot as plt
import diazdelser_fastatools as ft
import re
from tqdm import tqdm
import time
import random
import pickle

plt.style.use('dark_background')

In [19]:
# Files
small_distances_file = './data/small-distances.txt'
diistances_file = './data/distances.txt'

### Ex. 1 _((5+2 pts)_ Naive string matching

#### (a) _(5 pts)_ Mapping reads

Write a function ```map_reads_naive(reads,sequence)``` that takes a collection (e.g., a
list) of (short) strings, the reads, and identifies all occurring positions in sequence.
The function should return a dictionary, where the keys are those reads that occur in
the sequence and the value of a key is a list of positions (i.e., the indices) where the key
occurs.

E.g. Let 'abracadabra' be the sequence and ```['a','bra','cada','arba']``` be the list
of short reads. The function should return a dictionary:
```{'a': [0,3,5,7,10], 'bra': [1,8], 'cada': [4]}```


In [2]:
def map_reads_naive(reads:list,seq:str) -> dict:
	"""Takes a collection of (short) strings, the reads, and identifies all occurring positions in sequence"""
	results = {}
	for read in reads:
		res = [ match.span()[0] for match in re.finditer(re.compile(read), seq) ]
		# Remove empty lists
		if res:
			results[read] = res
	return results


In [21]:
# Test it
reads = ['a','bra','cada','arba']
seq = 'abracadabra'
matches = map_reads_naive(reads, seq)
matches

{'a': [0, 3, 5, 7, 10], 'bra': [1, 8], 'cada': [4]}

### _(Optional 2 pts)_ English words in the genome
The function can now, for instance, be used to solve the `vital' problem of identifying English words
in the proteome of organisms. Use the dictionary file words.txt and select all the English words with at
least four letters. Find the occurrences of English words of at least four letters in the proteomes of
Escherichia coli (file ```ecoli-proteome.faa```) and Drosophila melangoster (files ```chrX.faa```, ```chr2L.faa```,
```chr2R.faa```, ```chr3L.faa```, ```chr3R.faa```).

In [3]:
# Open words.txt file into list
def read_list(file:str) -> list:
	"""Create a "dictionary" list from words in a .txt file"""
	with open (file, 'r') as f:
		# filter by length >= 4
		return [word for word in f.read().split() if (len(word)>=4)]

english_words = read_list('words.txt')

In [4]:
# Open each fasta as string:
files_all = ['ecoli-proteome.faa', 'drosophila/chrX.faa', 'drosophila/chr2L.faa', 'drosophila/chr3L.faa',
		 'drosophila/chr2R.faa', 'drosophila/chr3R.faa']

In [9]:
file = ['ecoli-proteome-test.faa']

In [10]:
def find_english_words(files:list) ->dict:
	"""Find english words in fasta sequences"""
	results = {}
	for filename in tqdm(files):
		with open(filename, 'r') as f:
			results[filename] = {}
			g = ft.single_fasta_sequence(f)
			for header,sequence in g:
				reads = map_reads_naive(reads=english_words, seq=sequence)
				if reads:
					results[filename][header] = reads
	return results

In [12]:
results1 = find_english_words(file) # Short version: test
results = results1

100%|██████████| 1/1 [19:33<00:00, 1173.14s/it]


In [ ]:
# It takes about 12 hours per file (82 hours predicted) and I don't have that kind of time
# results2 = find_english_words(files_all)

In [ ]:
# If you run it, save it
# with open('results.pickle', 'wb') as f:
# 	pickle.dump(results2, f, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
# It it's saved, open it Open it
# with open ('results.pickle', 'rb') as f:
# 	results = pickle.load(f)


Answer the following questions:

	i. For Escherichia coli and Drosophila melangoster find the longest English words and the proteins in which
		they occur.

In [20]:
# Sort resulting dictionary by length of key
for organism, records, in results.items():
	# Get list of longest word for each record
	longest_record = { record : max(reads.keys(), key=len) for record, reads in records.items() }

	# Of that list, get overall longest for each organism
	longest = max(longest_record, key=longest_record.get)

	print(f"In {organism} the longest English word found is '{longest_record.get(longest)}' in {longest}.")


In ecoli-proteome-test.faa the longest English word found is 'aasvogels' in >gi|16128175|ref|NP_414724.1| tetraacyldisaccharide-1-P synthase [Escherichia coli str. K-12 substr. MG1655].


	ii. Find the proteins of Escherichia coli and Drosophila melangoster containing the most English words of at
		least four letters.

In [26]:
# Check lengths
# records={header: reads}
for organism, records in results.items():
	# Get list of number of words for each record (if words contain at least 4 letters
	num_records = { record : len([ x for x in reads.keys() if len(x)>3]) \
					   for record, reads in records.items() }

	most_records = max(records, key=longest_record.get)
	print(f'In {organism}:\nFound {num_records.get(most_records)} English words of at least four letters in {most_records}')

In ecoli-proteome-test.faa:
Found 3 English words of at least four letters in >gi|16128175|ref|NP_414724.1| tetraacyldisaccharide-1-P synthase [Escherichia coli str. K-12 substr. MG1655]


### Ex.2 _(6 pts)_ Speed of matching
Finding English words in a proteome is not very close to the practical problem of mapping reads of nucleotide sequences
to a genome. A slighlty more realistic scenario is to  use randomly generated data. Note, that for simulating
realistic data using simple random sequences is still not very good, as, of course, genomes are not random.
For realistic evaluation one would like to either use real experimental data or a more sophisticated randomized
model. However, for the assessment of mapping effciency, using simple random data is good enough for now.

#### (a) _(3 pts)_ Generate random data
Write a function ```get_random_data(genome_size,nr_of_reads,read_length,random_seed)```
which generates a random string over the alphabet ```{a, c, g, t}``` of length ```genome_size```
representing the genome. From this genome, randomly select ```nr_of_reads``` substrings
of length ```read_length``` comprising the set of reads. Use the parameter ```random_seed``` to
set up the random number generator, so the identical set of random sequences can be
reproduced. The function should return a tuple containing the sequence and the set of
reads.

In [32]:
def random_reads(sequence:str, read_length:int) -> str:
	"""Get one random read of given sequence"""
	i = random.randrange(0, len(sequence) - read_length + 1)
	return sequence[i : (i+read_length)]

def get_random_data(genome_size:int,nr_of_reads:int,read_length:int,random_seed:int=42) ->tuple:
	"""Generates a random genome"""
	random.seed(random_seed)
	sequence = ''.join(random.choices(['a','c','g','t'], k=genome_size))
	reads = [ random_reads(sequence,read_length) for i in range(nr_of_reads) ]
	return sequence, reads

#### (b) _(3 pts)_ Test performance of read mapping
Use random data to determine the running time of ```map_reads_naive``` of ```Ex.1.(a)``` for:

	i. A genome of size 10^6 and 10^4 reads of length 10.
	ii. A genome of size 10^6 and 10^5 reads of length 11.
	iii. A genome of size 10^7 and 10^4 reads of length 12.

Compare the running times to those of ```Ex. 3 (b)``` and ```4 (d)```.

Warning: ii. and iii. might take quite some time (around 5 min. - 60 min.) so you
might want to wait testing on these data sets until you finished everything.

In [33]:
# Test it
def test_it(genome_size:int,nr_of_reads:int,read_length:int):
	"""Checks running time of map_read_naive with random data """
	sequence, reads = get_random_data(genome_size,nr_of_reads,read_length)
	tic = time.perf_counter()
	map_reads_naive(reads,sequence)
	toc = time.perf_counter()
	print(f"\nGenome of size {genome_size} with {nr_of_reads} reads of length {read_length}.")
	print(f"Running time: {toc - tic:0.4f} seconds")

In [34]:
test_it(genome_size=1000000,nr_of_reads=10000,read_length=10)


Genome of size 1000000 with 10000 reads of length 10.
Running time: 75.1108 seconds


In [35]:
test_it(genome_size=1000000,nr_of_reads=100000,read_length=11)


Genome of size 1000000 with 100000 reads of length 11.
Running time: 475.6360 seconds


In [36]:
test_it(genome_size=10000000,nr_of_reads=10000,read_length=12)


Genome of size 10000000 with 10000 reads of length 12.
Running time: 506.1959 seconds


### Ex.3. _(9 pts)_ Preprocessing the "genome"
One idea for preprocessing is to keep a complete dictionary of all short reads of a fixed length
occurring in the genome. For instance, for the "genome" ```acgtatcgtatcc``` we would want our
dictionary of four-letter word to look like this:
```
{'acgt': [0],
'atcc': [9],
'atcg': [4],
'cgta': [1, 6],
'gtat': [2, 7],
'tatc': [3, 8],
'tcgt': [5]}
```

We can generate such a dictionary by scanning the genome sequence once, from left to right,
looking at the four letter sequences ```acgt, cgta, gtat,```... and adding the positions to the
appropriate dictionary entry. Now, if we want to look for a read (e.g., ```gtatcc```) we look up
the positions of the prefix of length 4 (e.g., ```gtat```) and we only need to check if the read
occurs at one of the positions indicated by the occurrences of the prefix (e.g., 2 and 7, where
the subsequence starting at 7 matches the read).

#### (a) _(6 pts)_ Implement the read mapping function ```map_reads_lookup(reads,sequence,lookup_size)``` using the outlined strategy.
Here, ```lookup_size``` is the size of the substrings to be used in the lookup table.
To make this function effcient it should preprocess the sequence once at the beginning
of the function and then use the generated lookup dictionary for finding all the reads.
It should return a dictionary as described in ex. 1 (a).

In [37]:
from collections import defaultdict
def reads_dictonary(seq:str, n:int) -> dict:
	"""Generates an n-sized-reads dictionary from the given sequence"""
	# Init dict list
	reads_dict = defaultdict(list)
	# Preprocess
	for i in range(len(seq)-n+1):
		reads_dict[seq[i:i+n]].append(i)
	return reads_dict


In [38]:
def lookup(lookup_dct, read, lookup_size, sequence):
	"""Check where the read starts with the lookup dictionary.
	  Check the rest with the sequence"""
	pos = lookup_dct.get(read[0:lookup_size])
	seqmatches = { sequence[p:p+len(read)] : p  for p in pos if pos }
	return [ pos for match, pos in seqmatches.items() if match == read ]

In [39]:
def map_reads_lookup(reads:list, sequence:str, lookup_size:int) -> dict:
	"""Finds all the reads in a sequence. Preprocesses the sequence once at the beginning
	 and then uses the generated lookup dictionary to find all the reads."""

	# Pre-process the seq one to create lookup dict
	lookup_dct = reads_dictonary(sequence, lookup_size)

	# Use lookup dict to find all the reads
	results = { read: lookup(lookup_dct, read, lookup_size, sequence) for read in reads if len(read)>=lookup_size }

	return results

#### (b) _(3 pts)_ Use a lookup size of 4 and 8 and determine the running times for the sample data generated in ex. 2 (b)

In [40]:
# Test it
def test_it2(genome_size:int,nr_of_reads:int,read_length:int, lookup_size:int):
	"""Checks running time of map_read_naive with random data """
	# Generate random data
	sequence, reads = get_random_data(genome_size,nr_of_reads,read_length)
	tic = time.perf_counter()
	map_reads_lookup(reads,sequence, lookup_size)
	toc = time.perf_counter()
	print(f"\nGenome of size {genome_size} with {nr_of_reads} reads of length {read_length}.")
	print(f"Running time: {toc - tic:0.4f} seconds")

In [41]:
test_it2(genome_size=1000000,nr_of_reads=10000,read_length=10, lookup_size=4)
test_it2(genome_size=1000000,nr_of_reads=10000,read_length=10, lookup_size=8)



Genome of size 1000000 with 10000 reads of length 10.
Running time: 24.2435 seconds

Genome of size 1000000 with 10000 reads of length 10.
Running time: 0.9436 seconds


In [42]:
test_it2(genome_size=1000000,nr_of_reads=100000,read_length=11,lookup_size=4)
test_it2(genome_size=1000000,nr_of_reads=100000,read_length=11,lookup_size=8)


Genome of size 1000000 with 100000 reads of length 11.
Running time: 254.1104 seconds

Genome of size 1000000 with 100000 reads of length 11.
Running time: 2.4117 seconds


In [43]:
test_it2(genome_size=10000000,nr_of_reads=10000,read_length=12,lookup_size=4)
test_it2(genome_size=10000000,nr_of_reads=10000,read_length=12,lookup_size=8)



Genome of size 10000000 with 10000 reads of length 12.
Running time: 339.5088 seconds

Genome of size 10000000 with 10000 reads of length 12.
Running time: 12.4669 seconds


### Ex.4 _(10 + 2 pts)_ Preprocessing the reads

#### (a) _(4 pts)_ Write a Python class KeywordTree for representing such a keyword tree.
The class will need to contain at least one method ```add_word```for adding new words to a tree.

A function for generating a keyword tree from a list of reads might then look like this:
```
def make_keyword_tree(reads):
	tree = KeywordTree()
	for word in reads:
	tree.add_word(word)
	return tree
```
(If you feel like it, consider adding an (optional) argument to the constructor so that
the call ```KeywordTree(reads)``` returns a keyword tree where all words in reads have
already been added.)

In [29]:
from collections import defaultdict

class KeywordTree():
	"""Represents a keyword tree"""
	def __init__(self, reads:list=None):
		self.root = {}
		self.reads = reads

		if self.reads:
			self.__add_reads()

	def __add_letter(self,parent,i,word):
		"""Add a letter to node"""
		letter = word[i]
		if parent == '*':
			return
		if i < len(word)-1:
			# If the letter is not added
			if not parent.get(letter):
				parent[letter] = {}
			self.__add_letter(parent=parent[letter],i=i+1, word=word)
		else:
			parent[letter] = '*'
			return

	def add_word(self,word):
		"""Add a word to the keyword tree"""
		self.__add_letter(self.root,0,word)

	def __add_reads(self):
		"""Add all given reads to the tree"""
		for word in self.reads:
			self.add_word(word)
		return

In [30]:
def make_keyword_tree(reads):
	tree = KeywordTree()
	for word in reads:
		tree.add_word(word)
	return tree.root

In [46]:
reads = ['a','bra','cada','arba']
make_keyword_tree(reads)

{'a': '*', 'b': {'r': {'a': '*'}}, 'c': {'a': {'d': {'a': '*'}}}}

In [47]:
reads = ['a','bra','cada','arba']
tree = KeywordTree(reads)
tree.root

{'a': '*', 'b': {'r': {'a': '*'}}, 'c': {'a': {'d': {'a': '*'}}}}

#### (b) _(3 pts)_ Write a read mapping function ```map_reads_tree(reads_tree,sequence)```

With such a keyword tree, it is easy to check whether a sequence starting at index ```i``` contains a word from the
keyword tree: Check if there is an edge from the root of the tree corresponding to the first letter ```sequence[i]```,
if so, proceed to ```i+1``` and to the corresponding child; repeat. If a child is marked as a word, the subsequence
```sequence[i:i+k+1]``` is a valid word. If at one point, no appropriate edge exist, stop.

The process is repeated for all i in the range 0 until len(sequence).

Write a read mapping function ```map_reads_tree(reads_tree,sequence)```, which uses
keyword trees for read mapping. Here, te argument reads_tree should be the keyword
tree generated from the reads. The function, again, should return a dictionary as
described in ex. 1 (a).

In [31]:
def is_edge(parent:dict,i:int, k:int, results:dict, seq:str) -> dict:
	"""Check if theres an edge from parent to child"""
	if i < len(seq):
		if not parent.get(seq[i]):
			return results

		if parent.get(seq[i]) == '*':
			results[seq[k:i+1]].append(k)
			return results

		parent.get(seq[i])
		results = is_edge(parent.get(seq[i]),i+1, k, results, seq)

	return results

In [32]:
def map_reads_tree(tree:object,seq:str) -> dict:
	"""Maps a read_tree"""
	# This allows us to append positions too each read more easily
	results = defaultdict(list)
	for i in range(len(seq)):
		results = is_edge(tree.root,i,i,results, seq)
	# Turn it back into a normal dict
	return dict(results)

In [50]:
reads = ['a','bra','cada','arba']
tree = KeywordTree(reads)
seq = 'abracadabra'
map_reads_tree(tree, seq)

# Should be : {'a': [0,3,5,7,10], 'bra': [1,8], 'cada': [4]} (CORRECT!)

{'a': [0, 3, 5, 7, 10], 'bra': [1, 8], 'cada': [4]}

####  (c) (Optional 2 pts)
Repeat the exercise 1 (b) of finding the longest English words in the proteomes of Escherichia coli and Drosophila
melangoster and compare the running times.


In [27]:
# Open each fasta as string:
files_all = ['ecoli-proteome.faa', 'drosophila/chrX.faa', 'drosophila/chr2L.faa', 'drosophila/chr3L.faa',
		 'drosophila/chr2R.faa', 'drosophila/chr3R.faa']

In [28]:
file = ['ecoli-proteome-test.faa']

In [35]:
def find_english_words(files:list) ->dict:
	"""Find english words in fasta sequences"""
	results = {}
	for filename in tqdm(files):
		with open(filename, 'r') as f:
			results[filename] = {}
			g = ft.single_fasta_sequence(f)
			for header,sequence in g:
				tree = KeywordTree(english_words)
				reads = map_reads_tree(tree=tree, seq=sequence)
				if reads:
					results[filename][header] = reads
	return results

In [36]:
results3 = find_english_words(file) # Short version: test

100%|██████████| 1/1 [02:08<00:00, 128.23s/it]


In [39]:
results4 = find_english_words(files_all) # Long version

100%|██████████| 6/6 [7:58:41<00:00, 4786.94s/it]  


In [45]:
results = results3
# Sort resulting dictionary by length of key
for organism, records, in results.items():
	# Get list of longest word for each record
	longest_record = { record : max(reads.keys(), key=len) for record, reads in records.items() }
	# Of that list, get overall longest for each organism
	longest = max(longest_record, key=longest_record.get)
	print(f"In {organism} the longest English word found is '{longest_record.get(longest)}' in {longest}.")

In ecoli-proteome-test.faa the longest English word found is 'aasvogel' in >gi|16128175|ref|NP_414724.1| tetraacyldisaccharide-1-P synthase [Escherichia coli str. K-12 substr. MG1655].


In [46]:
# Check lengths
# records={header: reads}
for organism, records in results.items():
	# Get list of number of words for each record (if words contain at least 4 letters
	num_records = { record : len([ x for x in reads.keys() if len(x)>3]) \
					   for record, reads in records.items() }

	most_records = max(records, key=longest_record.get)
	print(f'In {organism}:\nFound {num_records.get(most_records)} English words of at least four letters in {most_records}')


In ecoli-proteome-test.faa:
Found 2 English words of at least four letters in >gi|16128175|ref|NP_414724.1| tetraacyldisaccharide-1-P synthase [Escherichia coli str. K-12 substr. MG1655]


#### (d) (3 pts) Determine the running times for the sample data generated in ex. 2 (b).

In [51]:
# Test it
def test_it3(genome_size:int,nr_of_reads:int,read_length:int):
	"""Checks running time of map_read_naive with random data """
	# Generate random data
	sequence, reads = get_random_data(genome_size,nr_of_reads,read_length)

	tic = time.perf_counter()
	# Make Keyword tree
	tree = KeywordTree(reads)
	# Lookup
	map_reads_tree(tree,sequence)
	toc = time.perf_counter()

	print(f"\nGenome of size {genome_size} with {nr_of_reads} reads of length {read_length}.")
	print(f"Running time: {toc - tic:0.4f} seconds")


In [52]:
test_it3(genome_size=1000000,nr_of_reads=10000,read_length=10)


Genome of size 1000000 with 10000 reads of length 10.
Running time: 5.7910 seconds


In [53]:
test_it3(genome_size=1000000,nr_of_reads=100000,read_length=11)


Genome of size 1000000 with 100000 reads of length 11.
Running time: 9.0760 seconds


In [54]:
test_it3(genome_size=10000000,nr_of_reads=10000,read_length=12)


Genome of size 10000000 with 10000 reads of length 12.
Running time: 54.6013 seconds
